In [1]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out




# ------------------------------------- 在面试之前把这段代码背下来 ------------------------------------------------
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes: # 利用1x1卷积对齐高和宽 对齐输出通道数
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet50()
    print(net)
    y = net(torch.randn(4, 3, 32, 32))
    print(y.size())


In [2]:
#test()

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    # cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]
    # cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), (1024,1)]
    #cfg = [48, (96,2), 96, (192,2), 192, (384,2), 384, 384, 384, 384, 384, (768,2), (1024,1)]
    cfg = [64, (128,2), 128, 256, 256, (512,2), 512, 512, 512, 512, 512,1024,1024]
    
    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            if isinstance(x, int):
                out_planes = x
                stride = 1 
                layers.append(Block(in_planes, out_planes, stride))
            elif isinstance(x, tuple):
                out_planes = x[0]
                stride = x[1]
                layers.append(Block(in_planes, out_planes, stride))
            # AC层通过list存放设置参数
            elif isinstance(x, list):
                out_planes= x[0]
                stride = x[1] if len(x)==2 else 1
                layers.append(Block_Attention(in_planes, out_planes, stride))   
            else:
                pass
            
            in_planes = out_planes
            
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        #print("X",out.shape)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F



class BANUpdater(object):
    def __init__(self, **kwargs):
        self.model = kwargs.pop("model")
        self.optimizer = kwargs.pop("optimizer")
        self.n_gen = kwargs.pop("n_gen")
        self.last_model = None
        self.gen = 0

    def update(self, inputs, targets, criterion):
        self.optimizer.zero_grad()
        outputs = self.model(inputs)
        
        if self.gen > 0:
            teacher_outputs = self.last_model(inputs).detach()
            loss = self.kd_loss(outputs, targets, teacher_outputs)
        else:
            loss = criterion(outputs, targets)

        loss.backward()
        self.optimizer.step()
        return loss

    def register_last_model(self, weight):
        if weight=="":
            return
        # ------------------------------------ 匹配正确 --------------------------------------------------
        self.last_model =MobileNet(10)#ResNet18()# LeNet() #ResNet50() 
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = "cpu"
        self.last_model.to(device)
        
        state=torch.load(weight)
        self.last_model.load_state_dict(state)
        
        
    
    # ------------------------------------ 核心 Loss 计算 ------------------------------------------------
    def kd_loss(self, outputs, labels, teacher_outputs, alpha=0.9, T=20):
        KD_loss = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1),
                                 F.softmax(teacher_outputs/T, dim=1)) * (alpha * T * T) + \
                                 F.cross_entropy(outputs, labels) * (1. - alpha)

        return KD_loss

    def __model(self):
        return self.model

    def __last_model(self):
        return self.last_model

    def __gen(self):
        return self.gen


In [5]:
# -*- coding: utf-8 -*-


class Logger(object):
    def __init__(self, args):
        self.args = args

    def print_args(self):
        print("weight: ", self.args.weight)
        print("lr: ", self.args.lr)
        print("n_epoch: ", self.args.n_epoch)
        print("batch_size: ", self.args.batch_size)
        print("n_gen: ", self.args.n_gen)
        print("dataset: ", self.args.dataset)
        print("outdir: ", self.args.outdir)
        print("print_interval: ", self.args.print_interval)

    def print_log(self, epoch, it, train_loss, val_loss,accuracy):
        print("epoch: {}, iter: {}, train_loss: {}, test_loss: {},test accuracy: {}%".format(
            epoch, it, train_loss, val_loss,accuracy
        ))


In [6]:
# -*- coding: utf-8 -*-
import os
import argparse
import numpy as np

import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.optim.lr_scheduler import StepLR



class ArgParser(object):
    
    def __init__(self):
        
        self.weight=None
        self.lr=0.01
        self.n_epoch=110#100
        self.batch_size=256
        self.n_gen=4
        self.resume_gen=0
        self.dataset="cifar10"
        self.outdir="snapshots"
        self.print_interval=500
        
        
def main():

    args=ArgParser()

    logger = Logger(args)
    logger.print_args()

    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = "cpu"

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])

    trainset = CIFAR10(root="./data",
                       train=True,
                       download=True,
                       transform=transform)
    testset = CIFAR10(root="./data",
                      train=False,
                      download=True,
                      transform=transform)

    train_loader = DataLoader(trainset,
                              batch_size=args.batch_size,
                              shuffle=True,
                              num_workers=4)
    test_loader = DataLoader(testset,
                             batch_size=args.batch_size,
                             shuffle=False,
                             num_workers=4)
    print(len(test_loader))
    
    i = 0
   
    best_loss_list = []

    print("train...")
    last_model_weight=""
    for gen in range(args.resume_gen, args.n_gen):
        
        import gc
        if "model" in locals() or 'model' in globals():
            del model
            print("delete last model!!!")
        gc.collect()
        
        # From https://github.com/kuangliu/pytorch-cifar 
        if torch.cuda.is_available():
            model=MobileNet(10).cuda()
        else:
            model=MobileNet(10)
 
        optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = StepLR(optimizer, step_size=40, gamma=0.1)
        criterion = nn.CrossEntropyLoss()

        kwargs = {
            "model": model,
            "optimizer": optimizer,
            "n_gen": args.n_gen,
        }
        
        updater = BANUpdater(**kwargs)
        updater.register_last_model(last_model_weight)
        
        # -----------------------------------------------------------------------
        best_loss = 1e+9
        since=time.time()
        for epoch in range(args.n_epoch):
            
            # ---------------------
            train_loss = []
            for idx, (inputs, targets) in enumerate(train_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                t_loss = updater.update(inputs, targets, criterion).item() # the loss of this batch
                train_loss.append(t_loss)
                i += 1
            
            scheduler.step()
            
            # ---------------------
            val_loss = []
            correct=0
            with torch.no_grad():
                for idx, (inputs, targets) in enumerate(test_loader):
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = updater.model(inputs)
                    loss = criterion(outputs, targets).item()
                    val_loss.append(loss)
                    pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
                    correct += pred.eq(targets.data.view_as(pred)).sum()
            
            mean_val_loss = np.mean(val_loss)
            mean_train_loss=np.mean(train_loss)
            accuracy = 100. * correct.cpu().numpy() / len(test_loader.dataset)
            
            if mean_val_loss < best_loss:
                best_loss = mean_val_loss
                last_model_weight = os.path.join(args.outdir,"model"+str(gen)+".p")
                torch.save(updater.model.state_dict(),last_model_weight)
            
            print("escaped time of this epoch:{}".format(time.time()-since))
            logger.print_log(epoch, i, mean_train_loss, mean_val_loss,accuracy)

        print("best loss: ", best_loss)
        print("Born Again...")
        updater.gen += 1
        best_loss_list.append(best_loss)
        best_loss = 1e+9 # reset 

    for gen in range(args.n_gen):
        print("Gen: ", gen,
              ", best loss: ", best_loss_list[gen])


if __name__ == "__main__":
    main()


weight:  None
lr:  0.01
n_epoch:  110
batch_size:  256
n_gen:  4
dataset:  cifar10
outdir:  snapshots
print_interval:  500
Files already downloaded and verified
Files already downloaded and verified
40
train...
escaped time of this epoch:27.33865761756897
epoch: 0, iter: 196, train_loss: 1.5186141048158919, test_loss: 1.181091320514679,test accuracy: 57.26%
escaped time of this epoch:54.60605502128601
epoch: 1, iter: 392, train_loss: 0.9834637252651915, test_loss: 0.9323847070336342,test accuracy: 67.87%
escaped time of this epoch:81.70118522644043
epoch: 2, iter: 588, train_loss: 0.750357023915466, test_loss: 0.7779528856277466,test accuracy: 73.53%
escaped time of this epoch:108.90068888664246
epoch: 3, iter: 784, train_loss: 0.6249317137562499, test_loss: 0.6175604403018952,test accuracy: 79.02%
escaped time of this epoch:136.0862214565277
epoch: 4, iter: 980, train_loss: 0.5364558488434675, test_loss: 0.617492663115263,test accuracy: 79.81%
escaped time of this epoch:163.2630989551

escaped time of this epoch:1486.5013403892517
epoch: 53, iter: 10584, train_loss: 0.003963698982261121, test_loss: 0.4689834304153919,test accuracy: 89.33%
escaped time of this epoch:1513.6214578151703
epoch: 54, iter: 10780, train_loss: 0.003345476358900873, test_loss: 0.47207449227571485,test accuracy: 89.33%
escaped time of this epoch:1540.7544169425964
epoch: 55, iter: 10976, train_loss: 0.002992336605010288, test_loss: 0.472307950258255,test accuracy: 89.27%
escaped time of this epoch:1567.8727488517761
epoch: 56, iter: 11172, train_loss: 0.0028579318634595493, test_loss: 0.47587345764040945,test accuracy: 89.22%
escaped time of this epoch:1594.9918699264526
epoch: 57, iter: 11368, train_loss: 0.002739743695461324, test_loss: 0.47728667557239535,test accuracy: 89.25%
escaped time of this epoch:1622.0585527420044
epoch: 58, iter: 11564, train_loss: 0.002517741349316677, test_loss: 0.4775057226419449,test accuracy: 89.29%
escaped time of this epoch:1649.102724313736
epoch: 59, iter:

escaped time of this epoch:2923.995572090149
epoch: 106, iter: 20972, train_loss: 0.0017752002221437134, test_loss: 0.49920338317751883,test accuracy: 89.15%
escaped time of this epoch:2951.101505279541
epoch: 107, iter: 21168, train_loss: 0.0018409550028415968, test_loss: 0.49924135208129883,test accuracy: 89.19%
escaped time of this epoch:2978.2056651115417
epoch: 108, iter: 21364, train_loss: 0.0018040428736380168, test_loss: 0.4991818740963936,test accuracy: 89.18%
escaped time of this epoch:3005.3637115955353
epoch: 109, iter: 21560, train_loss: 0.0018515893809345303, test_loss: 0.4992800407111645,test accuracy: 89.14%
best loss:  0.36928629204630853
Born Again...
delete last model!!!
escaped time of this epoch:27.23776912689209
epoch: 0, iter: 21756, train_loss: 1.5386635624632543, test_loss: 1.2331116765737533,test accuracy: 55.54%
escaped time of this epoch:54.42804145812988
epoch: 1, iter: 21952, train_loss: 1.0386611265795571, test_loss: 0.9242540195584297,test accuracy: 67.8

escaped time of this epoch:1358.9301190376282
epoch: 49, iter: 31360, train_loss: 0.005992994477440204, test_loss: 0.47583772540092467,test accuracy: 89.19%
escaped time of this epoch:1386.1467733383179
epoch: 50, iter: 31556, train_loss: 0.005289831624024225, test_loss: 0.48092873468995095,test accuracy: 89.12%
escaped time of this epoch:1413.3036875724792
epoch: 51, iter: 31752, train_loss: 0.004594399807594565, test_loss: 0.483601126819849,test accuracy: 89.15%
escaped time of this epoch:1440.424166917801
epoch: 52, iter: 31948, train_loss: 0.004374461158235766, test_loss: 0.49057629331946373,test accuracy: 89.11%
escaped time of this epoch:1467.522168636322
epoch: 53, iter: 32144, train_loss: 0.004176663885805376, test_loss: 0.4948991656303406,test accuracy: 89.14%
escaped time of this epoch:1494.6705956459045
epoch: 54, iter: 32340, train_loss: 0.003352136679031715, test_loss: 0.4935164235532284,test accuracy: 89.15%
escaped time of this epoch:1521.83269405365
epoch: 55, iter: 325

escaped time of this epoch:2798.282855272293
epoch: 102, iter: 41748, train_loss: 0.0017396499336298024, test_loss: 0.5163183435797691,test accuracy: 89.16%
escaped time of this epoch:2825.3329322338104
epoch: 103, iter: 41944, train_loss: 0.0016750257262693985, test_loss: 0.5160164125263691,test accuracy: 89.14%
escaped time of this epoch:2852.476901292801
epoch: 104, iter: 42140, train_loss: 0.0017299981792552434, test_loss: 0.5161596842110157,test accuracy: 89.11%
escaped time of this epoch:2879.6430280208588
epoch: 105, iter: 42336, train_loss: 0.0017483028219727685, test_loss: 0.5160059414803981,test accuracy: 89.12%
escaped time of this epoch:2906.8298869132996
epoch: 106, iter: 42532, train_loss: 0.0017253106379197265, test_loss: 0.5159581288695335,test accuracy: 89.11%
escaped time of this epoch:2934.0401322841644
epoch: 107, iter: 42728, train_loss: 0.0018040065971982417, test_loss: 0.5158899493515492,test accuracy: 89.14%
escaped time of this epoch:2961.2029848098755
epoch: 1

escaped time of this epoch:1253.4649560451508
epoch: 45, iter: 52136, train_loss: 0.013168149371156278, test_loss: 0.43776816949248315,test accuracy: 89.08%
escaped time of this epoch:1280.8309481143951
epoch: 46, iter: 52332, train_loss: 0.009615151097579879, test_loss: 0.4476898044347763,test accuracy: 88.98%
escaped time of this epoch:1308.3355901241302
epoch: 47, iter: 52528, train_loss: 0.00788087870602553, test_loss: 0.4536604553461075,test accuracy: 89.13%
escaped time of this epoch:1335.7245075702667
epoch: 48, iter: 52724, train_loss: 0.005999254039013568, test_loss: 0.4628301620483398,test accuracy: 89.07%
escaped time of this epoch:1363.0452995300293
epoch: 49, iter: 52920, train_loss: 0.005333862965926528, test_loss: 0.4673816077411175,test accuracy: 89.13%
escaped time of this epoch:1390.3545637130737
epoch: 50, iter: 53116, train_loss: 0.004989138916039801, test_loss: 0.469623813778162,test accuracy: 89.05%
escaped time of this epoch:1417.7281601428986
epoch: 51, iter: 53

escaped time of this epoch:2702.7071409225464
epoch: 98, iter: 62524, train_loss: 0.001777634420194568, test_loss: 0.5070163302123547,test accuracy: 89.31%
escaped time of this epoch:2730.0011541843414
epoch: 99, iter: 62720, train_loss: 0.0017900510253954906, test_loss: 0.5069737985730172,test accuracy: 89.25%
escaped time of this epoch:2757.2641036510468
epoch: 100, iter: 62916, train_loss: 0.0016888193726748684, test_loss: 0.5072260648012161,test accuracy: 89.26%
escaped time of this epoch:2784.572830915451
epoch: 101, iter: 63112, train_loss: 0.0017046584552914208, test_loss: 0.5074964366853237,test accuracy: 89.28%
escaped time of this epoch:2811.901654481888
epoch: 102, iter: 63308, train_loss: 0.0016939723596917655, test_loss: 0.5078685790300369,test accuracy: 89.24%
escaped time of this epoch:2839.1234724521637
epoch: 103, iter: 63504, train_loss: 0.0016985241375977592, test_loss: 0.5075949765741825,test accuracy: 89.25%
escaped time of this epoch:2866.480643749237
epoch: 104, 

escaped time of this epoch:1147.6354234218597
epoch: 41, iter: 72912, train_loss: 0.11400213216108326, test_loss: 0.3643701419234276,test accuracy: 88.99%
escaped time of this epoch:1174.944254398346
epoch: 42, iter: 73108, train_loss: 0.05064123378572415, test_loss: 0.37366200722754,test accuracy: 89.18%
escaped time of this epoch:1202.2271800041199
epoch: 43, iter: 73304, train_loss: 0.029406244937824656, test_loss: 0.39011774137616156,test accuracy: 89.29%
escaped time of this epoch:1229.5242726802826
epoch: 44, iter: 73500, train_loss: 0.01924793219802027, test_loss: 0.40473023019731047,test accuracy: 89.43%
escaped time of this epoch:1256.877123117447
epoch: 45, iter: 73696, train_loss: 0.013528326925422465, test_loss: 0.4154531601816416,test accuracy: 89.23%
escaped time of this epoch:1284.1357870101929
epoch: 46, iter: 73892, train_loss: 0.01012238838747904, test_loss: 0.4276249945163727,test accuracy: 89.24%
escaped time of this epoch:1311.3620903491974
epoch: 47, iter: 74088, 

escaped time of this epoch:2598.0662713050842
epoch: 94, iter: 83300, train_loss: 0.0018722825833330198, test_loss: 0.48503536358475685,test accuracy: 89.19%
escaped time of this epoch:2625.3931515216827
epoch: 95, iter: 83496, train_loss: 0.0018458211181533275, test_loss: 0.4852078050374985,test accuracy: 89.2%
escaped time of this epoch:2652.7343492507935
epoch: 96, iter: 83692, train_loss: 0.001872966085009429, test_loss: 0.48537288084626196,test accuracy: 89.2%
escaped time of this epoch:2680.128867149353
epoch: 97, iter: 83888, train_loss: 0.0018096487061595734, test_loss: 0.4856234207749367,test accuracy: 89.21%
escaped time of this epoch:2707.5683732032776
epoch: 98, iter: 84084, train_loss: 0.0019971377986046125, test_loss: 0.48550191447138785,test accuracy: 89.2%
escaped time of this epoch:2734.958096265793
epoch: 99, iter: 84280, train_loss: 0.0018283173807763628, test_loss: 0.48563967272639275,test accuracy: 89.24%
escaped time of this epoch:2762.239415168762
epoch: 100, ite

In [13]:
import numpy as np

f1=open("Train_Info.txt","r")
num=0
res=[]
for aLine in f1:
    num=num+1
    tmp=[str(e) for e in aLine.strip("\n").split(" ")]
    if tmp[0]=="epoch:" and num>180:
        res.append(float(tmp[-1].strip("%")))
    
f1.close()
print(np.mean(res))

89.2055


In [ ]:
# gen 1: 89.17050000000002

# gen 2: 89.17399999999999

# gen 3: 89.245

# 